<a href="https://colab.research.google.com/github/dv0310/-Dilip-Vatnala_INFO5731_Fall2023/blob/main/Vatnala_Dilip_Goud_Exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [37]:
# Import necessary libraries
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import pandas as pd

# Define request headers for making web requests
Headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

# Initialize empty lists to store review titles and text
titles = []
text = []

# Loop through pages to scrape Amazon product reviews
for pg in range(1, 4000):
    # Define the URL for the Amazon product reviews page
    url = 'https://www.amazon.com/product-reviews/B08PNKRPCY/ref=acr_dp_hist_5?ie=UTF8&filterByStar=five_star&reviewerType=all_reviews'

    # Send a GET request to the URL with custom headers
    pg = requests.get(url, headers=Headers)

    # Parse the HTML content of the page using BeautifulSoup
    s = BeautifulSoup(pg.content, 'html.parser')

    # Find all review titles and add them to the 'titles' list
    reviews = s.find_all('a', class_='review-title-content')
    for i in range(len(reviews)):
        titles.append(reviews[i].get_text().strip())

    # Find all review text and add them to the 'text' list
    reviews = s.find_all('span', class_='review-text-content')
    for r in range(0, len(reviews)):
        text.append(reviews[r].get_text().strip())

# Create a DataFrame to store the review titles and text
reviews_df = pd.DataFrame({
    'Title': titles,
    'Review': text
})

# Save the DataFrame as a CSV file
reviews_df.to_csv('reviews.csv', index=False)

# Read the saved CSV file and print the first few rows
data_1 = pd.read_csv('reviews.csv')
print(data_1.head())

# Print the shape of the DataFrame
print(f'Shape of the data frame: {reviews_df.shape}')

# Download the CSV file to the local machine
from google.colab import files
files.download('reviews.csv')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



                                               Title  \
0  5.0 out of 5 stars\nexcellent condition phone ...   
1  5.0 out of 5 stars\nThe important things to kn...   
2                   5.0 out of 5 stars\nGreat phone!   
3                       5.0 out of 5 stars\nPerfect!   
4                       5.0 out of 5 stars\nLove it!   

                                              Review  
0  phone arrived in excellent external conditions...  
1  What I like about this iPhone is how to take c...  
2  Have only had it a couple months now but I hav...  
3  I read past reviews before purchase and was re...  
4  This is my first time ordering Amazon Renewed....  
Shape of the data frame: (1140, 2)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
# Read the data from the 'reviews.csv' file into a DataFrame
data_1 = pd.read_csv('reviews.csv')

# Display the first few rows of the DataFrame to inspect the data
data_1.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Title,Review
0,5.0 out of 5 stars\nexcellent condition phone ...,phone arrived in excellent external conditions...
1,5.0 out of 5 stars\nThe important things to kn...,What I like about this iPhone is how to take c...
2,5.0 out of 5 stars\nGreat phone!,Have only had it a couple months now but I hav...
3,5.0 out of 5 stars\nPerfect!,I read past reviews before purchase and was re...
4,5.0 out of 5 stars\nLove it!,This is my first time ordering Amazon Renewed....


In [39]:
import re

# Remove punctuation from the 'Review' column
data_1['Review_processed'] = data_1['Review'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the processed 'Review' text to lowercase
data_1['Review_processed'] = data_1['Review_processed'].map(lambda x: x.lower())

# Print the first few rows of the processed 'Review' text
data_1['Review_processed'].head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

<>:4: DeprecationWarning:

invalid escape sequence '\.'

<>:4: DeprecationWarning:

invalid escape sequence '\.'

<ipython-input-39-59f325892645>:4: DeprecationWarning:

invalid escape sequence '\.'



0    phone arrived in excellent external conditions...
1    what i like about this iphone is how to take c...
2    have only had it a couple months now but i hav...
3    i read past reviews before purchase and was re...
4    this is my first time ordering amazon renewed ...
Name: Review_processed, dtype: object

In [40]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define a list of stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Function to tokenize sentences into words
def sent_to_words(sentences):
    for sentence in sentences:
        # simple_preprocess tokenizes and removes punctuation
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Function to remove stopwords from a list of words
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

# Get the 'Review_processed' column from the DataFrame and convert it to a list
data = data_1.Review_processed.values.tolist()

# Tokenize the text data into words
data_words = list(sent_to_words(data))

# Remove stopwords from the tokenized text
data_words = remove_stopwords(data_words)

# Print the first 30 words of the preprocessed text in the first document
print(data_words[:1][0][:30])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['phone', 'arrived', 'excellent', 'external', 'conditions', 'small', 'slight', 'scratch', 'right', 'side', 'everything', 'worked', 'including', 'charging', 'face', 'id', 'camera', 'however', 'battery', 'arrived', 'less', 'suppliers', 'guaranteed', 'fortunately', 'paid', 'battery', 'swap', 'phone', 'excellent', 'customer']


In [41]:
import gensim.corpora as corpora

# Create a dictionary that maps words to unique IDs
id2word = corpora.Dictionary(data_words)

# Create a corpus, which is a bag of words representation
texts = data_words

# Compute the Term Document Frequency (TF-IDF) for the corpus
corpus = [id2word.doc2bow(text) for text in texts]

# Print the first 30 items of the TF-IDF representation of the first document
print(corpus[:1][0][:30])


[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [42]:
from pprint import pprint

# Define the number of topics
num_topics = 10

# Build an LDA model using the corpus and dictionary
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)

# Print the keywords associated with the topics
print(lda_model.print_topics())

# Apply the LDA model to the corpus to get topic distributions for each document
doc_lda = lda_model[corpus]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



[(0, '0.054*"phone" + 0.032*"one" + 0.024*"new" + 0.020*"mini" + 0.018*"iphone" + 0.017*"condition" + 0.016*"battery" + 0.013*"old" + 0.013*"time" + 0.013*"went"'), (1, '0.027*"phone" + 0.027*"lo" + 0.027*"funciona" + 0.026*"de" + 0.025*"never" + 0.019*"perfect" + 0.016*"new" + 0.014*"eso" + 0.014*"ha" + 0.014*"es"'), (2, '0.038*"people" + 0.030*"work" + 0.022*"iphone" + 0.022*"others" + 0.021*"forget" + 0.020*"want" + 0.019*"like" + 0.018*"right" + 0.018*"phone" + 0.017*"quality"'), (3, '0.038*"phone" + 0.032*"people" + 0.024*"iphone" + 0.022*"like" + 0.020*"time" + 0.019*"work" + 0.018*"great" + 0.016*"take" + 0.016*"right" + 0.014*"want"'), (4, '0.039*"battery" + 0.039*"time" + 0.039*"condition" + 0.038*"excellent" + 0.035*"little" + 0.034*"still" + 0.034*"better" + 0.034*"renewed" + 0.033*"though" + 0.033*"amazon"'), (5, '0.029*"time" + 0.021*"little" + 0.019*"love" + 0.019*"people" + 0.018*"condition" + 0.018*"amazon" + 0.018*"battery" + 0.017*"others" + 0.017*"renewed" + 0.017*"t

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [43]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, strip_short, stem_text

# Define a preprocessing function that cleans the text based on custom filters
def preprocess(text):

    # Define custom filters to clean the text
    CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_punctuation, strip_short, stem_text]

    # Preprocess the text using the custom filters
    text = preprocess_string(text, CUSTOM_FILTERS)

    return text

# Apply the preprocessing function to all reviews in the 'Review' column
data_1['Review_Text (Clean)'] = data_1['Review'].apply(lambda x: preprocess(x))

# Display the first few rows of the dataset with the cleaned text
data_1.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Title,Review,Review_processed,Review_Text (Clean)
0,5.0 out of 5 stars\nexcellent condition phone ...,phone arrived in excellent external conditions...,phone arrived in excellent external conditions...,"[phone, arriv, excel, extern, condit, onli, sm..."
1,5.0 out of 5 stars\nThe important things to kn...,What I like about this iPhone is how to take c...,what i like about this iphone is how to take c...,"[like, iphon, care, expens, luxuri, iphon, con..."
2,5.0 out of 5 stars\nGreat phone!,Have only had it a couple months now but I hav...,have only had it a couple months now but i hav...,"[coupl, month, it’, great, littl, phone, prett..."
3,5.0 out of 5 stars\nPerfect!,I read past reviews before purchase and was re...,i read past reviews before purchase and was re...,"[read, past, review, purchas, worri, come, bro..."
4,5.0 out of 5 stars\nLove it!,This is my first time ordering Amazon Renewed....,this is my first time ordering amazon renewed ...,"[time, order, amazon, renew, took, receiv, phi..."


In [44]:
from gensim import corpora

# Create a dictionary with the preprocessed text data
corpus = data_1['Review_Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# Convert the corpus into a bag of words representation
bow = [dictionary.doc2bow(text) for text in corpus]

from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Calculate the coherence score for different numbers of topics
for i in range(2, 11):
    # Create an LSI model with the specified number of topics
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)

    # Calculate the coherence score using the 'c_v' coherence measure
    coherence_model = CoherenceModel(model=lsi, texts=data_1['Review_Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # Print the coherence score for the current number of topics
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning:

Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/lo

Coherence score with 2 clusters: 0.9027459917452104


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning:

Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.



Coherence score with 3 clusters: 0.8311680628273864


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning:

Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.



...

Coherence score with 8 clusters: 0.9331292765654591


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning:

Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.



Coherence score with 9 clusters: 0.7107415390511446


/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning:

Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.



Coherence score with 10 clusters: 0.7511154902739126


In [45]:
# Create an LSI model with 2 topics
lsi = LsiModel(bow, num_topics=2, id2word=dictionary)

# Print the 5 words with the strongest association to each derived topic
for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in Topic {}: {}.'.format(topic_num, words))


Words in Topic 0: 0.388*"peopl" + 0.332*"don" + 0.229*"work" + 0.225*"you" + 0.221*"life".
Words in Topic 1: 0.551*"phone" + 0.242*"mini" + 0.235*"new" + 0.162*"went" + 0.162*"upgrad".


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:963: DeprecationWarning:

Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/local/lib/python3.10/dist-packages/scipy/linalg/_special_matrices.py:154: DeprecationWarning:

'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.

/usr/lo

In [46]:
# Apply the LSI model to the bag-of-words representation
corpus_lsi = lsi[bow]

# Initialize lists to store the topic scores for each review
score1 = []
score2 = []

# Extract topic scores from the LSI results
for doc in corpus_lsi:
    if len(doc) > 0 and len(doc[0]) > 1:
        score1.append(round(doc[0][1], 2))
    if len(doc) > 1 and len(doc[1]) > 1:
        score2.append(round(doc[1][1], 2))

# Create a DataFrame that shows scores assigned for both topics for each review
# Remove any reviews that were not processed by the LDA model
data_1 = data_1.iloc[:len(score1)]

# Create a DataFrame with the review text and topic scores
topic_df = pd.DataFrame()
topic_df['Text'] = data_1['Review']
topic_df['Topic 0 score'] = score1
topic_df['Topic 1 score'] = score2

# Determine the topic with the highest score for each review
topic_df['Topic'] = topic_df[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.argmax(), axis=1)

# Display the first few rows of the DataFrame
topic_df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Text,Topic 0 score,Topic 1 score,Topic
0,phone arrived in excellent external conditions...,1.25,2.20,1
1,What I like about this iPhone is how to take c...,17.17,-2.44,0
2,Have only had it a couple months now but I hav...,1.28,1.96,1
3,I read past reviews before purchase and was re...,0.57,0.96,1
4,This is my first time ordering Amazon Renewed....,0.40,0.59,1


In [47]:
# Create separate dataframes for each topic
topic_df0 = topic_df[topic_df['Topic'] == 0]
topic_df1 = topic_df[topic_df['Topic'] == 1]

# Print a random sample text from each topic (using a random seed for reproducibility)
print('Sample text from Topic 0:\n{}'.format(topic_df0.sample(1, random_state=2)['Text'].values))
print('Sample text from Topic 1:\n{}'.format(topic_df1.sample(1, random_state=2)['Text'].values))


Sample text from Topic 0:
['This is my first time ordering Amazon Renewed. It took a while to receive it in the Philippines but I love it! The battery though is 83% I was expecting a little better because I bought the "excellent condition" but I\'m still happy about it. Thank you again.']
Sample text from Topic 1:
["phone arrived in excellent external conditions (only a small slight scratch on the right side). everything worked including charging, face ID, and the camera. however, the battery arrived at 76% which is less than the suppliers guaranteed. fortunately they paid for my battery swap for the phone. excellent customer service and no hassle to get this done. 10/10 recommended if you're hesitant."]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [51]:
# Write your code here
!pip install pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [53]:
import nltk
nltk.download('all')
!pip install preprocess
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_r

In [54]:
import pyLDAvis
pyLDAvis.enable_notebook()

# Assuming you already have a topic model (LDA or LSI) stored in 'lda_model'

# Define the number of top words you want to extract for each topic
top = 10
topic_to_topwords = {}

# Loop through each topic
for j in range(num_topics):  # Replace 'num_topics' with the actual number of topics
    # Get the top words for the current topic from the topic model
    top_words = lda_model.show_topic(j, topn=top)

    # Extract the words and scores for the top words
    top_words_list = [word for word, score in top_words]

    # Create a message that shows the topic and its top words
    msg = f'Topic {j} has top words: {" ".join(top_words_list)}'
    print(msg)

    # Store the top words for this topic
    topic_to_topwords[j] = top_words_list


Topic 0 has top words: phone one new mini iphone condition battery old time went
Topic 1 has top words: phone lo funciona de never perfect new eso ha es
Topic 2 has top words: people work iphone others forget want like right phone quality
Topic 3 has top words: phone people iphone like time work great take right want
Topic 4 has top words: battery time condition excellent little still better renewed though amazon
Topic 5 has top words: time little love people condition amazon battery others renewed thank
Topic 6 has top words: phone iphone like pretty new one great time little mini
Topic 7 has top words: day phone battery stars condition smoothly literally arrival unlocked super
Topic 8 has top words: condition new like still renewed well two working ordered tell
Topic 9 has top words: excellent battery arrived phone right get customer conditions less guaranteed


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [55]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Define a list of English stopwords
stop_words = stopwords.words('english')

# Define a function to clean text
def clean_text(x):
    # Convert text to lowercase
    x = str(x).lower()

    # Remove hashtags (e.g., #example)
    x = re.sub(r'#[A-Za-z0-9]*', ' ', x)

    # Remove URLs (e.g., http://example.com)
    x = re.sub(r'https*://.*', ' ', x)

    # Remove mentions (e.g., @username)
    x = re.sub(r'@[A-Za-z0-9]+', ' ', x)

    # Tokenize the text and remove stopwords
    tokens = word_tokenize(x)
    x = ' '.join([w for w in tokens if not w.lower() in stop_words])

    # Remove special characters, digits, and multiple spaces
    x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
    x = re.sub(r'\d+', ' ', x)
    x = re.sub(r'\n+', ' ', x)
    x = re.sub(r'\s{2,}', ' ', x)

    return x

# Apply the clean_text function to the 'Review' column and store the cleaned text in a new column 'clean_text'
data_1['clean_text'] = data_1.Review.apply(clean_text)

# Display the first few rows of the DataFrame with the cleaned text
data_1.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

<>:32: DeprecationWarning:

invalid escape sequence '\('

<>:32: DeprecationWarning:

invalid escape sequence '\('

<ipython-input-55-ed9d1b4dd02c>:32: DeprecationWarning:

invalid escape sequence '\('

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Title,Review,Review_processed,Review_Text (Clean),clean_text
0,5.0 out of 5 stars\nexcellent condition phone ...,phone arrived in excellent external conditions...,phone arrived in excellent external conditions...,"[phone, arriv, excel, extern, condit, onli, sm...",phone arrived excellent external conditions sm...
1,5.0 out of 5 stars\nThe important things to kn...,What I like about this iPhone is how to take c...,what i like about this iphone is how to take c...,"[like, iphon, care, expens, luxuri, iphon, con...",like iphone take care expensive luxury iphone ...
2,5.0 out of 5 stars\nGreat phone!,Have only had it a couple months now but I hav...,have only had it a couple months now but i hav...,"[coupl, month, it’, great, littl, phone, prett...",couple months say great little phone pretty mu...
3,5.0 out of 5 stars\nPerfect!,I read past reviews before purchase and was re...,i read past reviews before purchase and was re...,"[read, past, review, purchas, worri, come, bro...",read past reviews purchase really worried woul...
4,5.0 out of 5 stars\nLove it!,This is my first time ordering Amazon Renewed....,this is my first time ordering amazon renewed ...,"[time, order, amazon, renew, took, receiv, phi...",first time ordering amazon renewed took receiv...


In [56]:
!pip install --upgrade joblib

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [57]:
! pip install bertopic

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.1
    Uninstalling numpy-1.26.1:
      Successfully uninstalled numpy-1.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires numpy>=1.24.2, but you have numpy 1.23.5 which is incompatible.


In [58]:
import pandas as pd

# Load your data into a DataFrame
data_frame = pd.read_csv('reviews.csv')  # Replace 'your_data.csv' with the actual path to your data file

# List the column names of the DataFrame
print(data_frame.columns)


Index(['Title', 'Review'], dtype='object')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [59]:
import pandas as pd

# Load your data into a DataFrame
data_frame = pd.read_csv('reviews.csv')  # Replace 'your_data.csv' with the actual path to your data file

# Now you can access your data from the 'data_frame' variable
reviews = data_frame['Title'].to_list()
timestamps = data_frame['Review'].to_list()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [60]:
from bertopic import BERTopic

reviews= data_frame.Title.to_list()
timestamp = data_frame.Review.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(reviews)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [61]:
topic_model.visualize_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [62]:
# Write your answer here (no code needed for this question)
"""
Topic modeling is a widely-adopted technique used to uncover themes or subjects within a set of documents.
 One way to assess the quality of a topic model is by examining the concept of Topic Coherence.
 Various techniques are available for topic modeling, including Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), as well as more recent methodologies like lda2vec and BERTopic.
 In the case of LDA, the model takes a dictionary and a corpus as inputs, in addition to specifying the desired number of topics. Visualization tools such as pyLDAvis can be employed to compare coherence scores for different topic quantities, aiding in the selection of the optimal number of topics.
 Generally, the goal is to identify the number of topics that maximizes the coherence score while still generating meaningful and interpretable topics. LSA is frequently utilized for dimensionality reduction or noise reduction in data.
 Topic Coherence can also be used to determine the optimal number of topics for an LSA model, with a higher coherence score indicating a better topic model. Comparing the coherence scores of LDA and LSA, it becomes evident that LDA typically yields superior results. To sum it up, both LDA and LSA serve as viable options for topic modeling, but LDA tends to outperform LSA in terms of Topic Coherence.
  The evaluation of a topic model's effectiveness involves comparing coherence scores for different topic quantities and selecting the model with the highest coherence score while maintaining the production of meaningful topics."""



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



"\nTopic modeling is a widely-adopted technique used to uncover themes or subjects within a set of documents.\n One way to assess the quality of a topic model is by examining the concept of Topic Coherence. \n Various techniques are available for topic modeling, including Latent Semantic Analysis (LSA), Latent Dirichlet Allocation (LDA), as well as more recent methodologies like lda2vec and BERTopic. \n In the case of LDA, the model takes a dictionary and a corpus as inputs, in addition to specifying the desired number of topics. Visualization tools such as pyLDAvis can be employed to compare coherence scores for different topic quantities, aiding in the selection of the optimal number of topics. \n Generally, the goal is to identify the number of topics that maximizes the coherence score while still generating meaningful and interpretable topics. LSA is frequently utilized for dimensionality reduction or noise reduction in data. \n Topic Coherence can also be used to determine the opt